In [1]:
import os

In [3]:
## ML model only for long words
import re
import collections
import pickle
import time
import numpy as np
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
import requests


class HangmanAI1:
    def __init__(self, access_token=None, session=None, timeout=None):
        self.hangman_url = self.determine_hangman_url()
        self.access_token = access_token
        self.session = session or requests.Session()
        self.timeout = timeout

        self.guessed_letters = []

        full_dictionary_location = "/kaggle/input/words-250000/words_250000_train.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)
        self.vowel_prior = self.initialize_vowel_prior()
        self.ngram_probs = self.build_ngram_probabilities(self.full_dictionary)  # New line


        # self.x, self.y = self.generate_data(self.full_dictionary)


        self.current_dictionary = []
        self.prev_values = [0, 0, 0]
        self.guessed_letters = []

        self.x, self.y = self.generate_data(self.full_dictionary)

        # Initialize models
        self.model_xgb = XGBClassifier(n_estimators=150, max_depth=9, subsample=0.7, colsample_bytree=0.7)
        self.model_lgb = LGBMClassifier(num_leaves=50, n_estimators=200, min_child_weight=0.5, reg_alpha=0.01, reg_lambda=0.01)
        self.model_cb = CatBoostClassifier(iterations=100, learning_rate=0.1, depth=2, loss_function='MultiClass', verbose=False)

        # Train or load models
        self.model_cb = self.training(self.model_cb, "catb_newc1.pkl")
        self.model_xgb = self.training(self.model_xgb, "xgboost_newc1.pkl")
        self.model_lgb = self.training(self.model_lgb, "lightgbm_newc1.pkl")

    @staticmethod
    def determine_hangman_url():
        links = ['https://trexsim.com', 'https://sg.trexsim.com']
        data = {link: 0 for link in links}
        for link in links:
            requests.get(link)
            for i in range(10):
                start = time.time()
                requests.get(link)
                data[link] += time.time() - start
        link = sorted(data.items(), key=lambda x: x[1])[0][0]
        return f"{link}/trexsim/hangman"
    def build_ngram_probabilities(self, dictionary, n=2):
        """
        Build n-gram probabilities from the word dictionary.
        """
        ngram_counts = collections.Counter()
        total_counts = collections.Counter()

        for word in dictionary:
            for i in range(len(word) - n + 1):
                ngram = word[i:i + n]
                ngram_counts[ngram] += 1
                total_counts[word[i]] += 1

        ngram_probs = {ngram: count / total_counts[ngram[0]] for ngram, count in ngram_counts.items()}
        return ngram_probs

    def build_dictionary(self, file_path):
        with open(file_path, "r") as f:
            return [line.strip() for line in f.readlines()]
    def training(self, classifier, weight_file):
        """Train or load the model"""
        model_path = os.path.join("/kaggle/input/gboosting/pytorch/default/1", weight_file)
        
        # Check if model file exists
        if os.path.exists(model_path):
            # If model exists, load it
            with open(model_path, 'rb') as f:
                model = pickle.load(f)
            print(f"Model loaded from {model_path}")
        else:
            # If model does not exist, train it
            model = classifier.fit(self.x, self.y)
            # Save the trained model
            with open(model_path, 'wb') as f:
                pickle.dump(model, f)
            print(f"Model trained and saved to {model_path}")

        return model
    def generate_data(self, dictionary):
        """
        Generate the feature set (self.x) and target labels (self.y) from the given word dictionary.
        Features are one-hot encoded word variations, and targets are the index of missing letters.
        
        Parameters:
        - dictionary: List of words used for training the models.
        
        Returns:
        - x: Feature array (one-hot encoded variations of words).
        - y: Label array (corresponding indices of missing letters).
        """
        x, y = [], []
        for word in dictionary:
            variations, missing_letters = self.create_variations(word)
            for variation, missing_letter in zip(variations, missing_letters):
                # Step 1: Alphabet encoding of word variation
                alphabet_encoded_variation = self.alphabet_encoding(variation)
                
                # Step 2: One-hot encoding of alphabetic positions
                onehot_encoded_variation = self.onehot_encoding(alphabet_encoded_variation)
                
                # Append the feature and the label to x and y lists respectively
                x.append(onehot_encoded_variation)
                
                # The label is the index of the missing letter in the alphabet (0-25)
                y.append(ord(missing_letter) - ord('a'))
        
        # Convert x and y into numpy arrays
        return np.array(x), np.array(y)
    

    def create_variations(self, word):
        variations, letters = [], []
        for i, char in enumerate(word):
            variation = word[:i] + '_' + word[i+1:]
            if variation not in variations:
                variations.append(variation)
                letters.append(char)
        return variations, letters

    def alphabet_encoding(self, word):
        return [ord(letter) - ord('a') + 1 for letter in word]

    def onehot_encoding(self, vector, max_len=30):
        padded_vector = np.zeros(max_len, dtype=int)
        padded_vector[:len(vector)] = vector
        return padded_vector

    def next_letter_prediction(self, encoding, model):
        predictions = model.predict_proba(encoding.reshape(1, -1))
        for i, prob in enumerate(predictions[0]):
            if chr(i + ord('a')) in self.guessed_letters:
                predictions[0][i] = 0
        return predictions

    def probability_max(self, predictions):
    # Weighted averaging of predictions
        weights = [0.4, 0.3, 0.3]  # Example weights for CatBoost, XGBoost, and LightGBM
        weighted_predictions = sum(weight * predictions[i][0] for i, weight in enumerate(weights))
        return np.array(weighted_predictions)

    def initialize_vowel_prior(self):
        """
        Initialize vowel prior probabilities based on word length.
        """
        vowel_counts = {length: collections.Counter() for length in range(1, 36)}  # Assuming max length of 35
        vowels = "aeiou"

        for word in self.full_dictionary:
            word_length = len(word)
            if word_length <= 35:
                for char in word:
                    if char in vowels:
                        vowel_counts[word_length][char] += 1

        vowel_prior = {}
        for length, counts in vowel_counts.items():
            total = sum(counts.values())
            if total > 0:
                vowel_prior[length] = {vowel: count / total for vowel, count in counts.items()}
            else:
                vowel_prior[length] = {vowel: 0 for vowel in vowels}

        return vowel_prior

    def guess(self, word, tries_remains):
        word = word.replace(" ", "")
        
        # Step 1: Check if word length is 6 or less
        if len(word.replace("_", "")) <= 6:
            # If word length is 6 or less, use only dictionary-based filtering and vowel prior
            final_predictions = self.refine_with_dictionary(word)
    
            # Step 2: If no letters are possible from the dictionary, use vowel prior or fallback guess
            if not final_predictions:
                if self.should_prefer_vowels():
                    return self.fallback_guess(word)
                else:
                    # If fallback mechanism is not applicable, return the best guess based on frequency
                    return self.fallback_guess(word)
    
            guessed_letter = final_predictions[0]  # Select the most frequent candidate
            self.guessed_letters.append(guessed_letter)
            return guessed_letter
    
        else:
            # If word length is more than 6, use machine learning models (XGBoost, CatBoost, LightGBM)
            encoding = self.onehot_encoding(self.alphabet_encoding(word))
    
            # Get predictions from ML models
            predictions = [
                self.next_letter_prediction(encoding, model)
                for model in [self.model_cb, self.model_xgb, self.model_lgb]
            ]
            
            final_predictions = self.probability_max(predictions)
    
            # Step 3: Refine predictions using dictionary
            possible_letters = self.refine_with_dictionary(word)
            if possible_letters:
                for i in range(26):
                    if chr(i + ord('a')) not in possible_letters:
                        final_predictions[i] = 0
    
            # Step 4: Normalize final predictions
            final_predictions /= final_predictions.sum()
    
            # Step 5: Integrate heuristics - Vowel preference if tries remaining > 4
            if self.should_prefer_vowels():
                for vowel in "aeiou":
                    idx = ord(vowel) - ord('a')
                    final_predictions[idx] *= 1.5
    
            # Step 6: Backup mechanism if no predictions
            if final_predictions.sum() == 0:
                return self.fallback_guess(word)
    
            # Step 7: Select the best guess
            guessed_letter = chr(final_predictions.argmax() + ord('a'))
            self.guessed_letters.append(guessed_letter)
            return guessed_letter


    def refine_with_dictionary(self, word):
        pattern = "^" + word.replace("_", ".") + "$"
        refined_dict = [w for w in self.full_dictionary if re.match(pattern, w)]
        
        # Score candidates based on frequency
        letter_frequencies = collections.Counter("".join(refined_dict))
        possible_letters = set()
        for candidate in refined_dict:
            for char in candidate:
                if char not in self.guessed_letters:
                    possible_letters.add((char, letter_frequencies[char]))
        
        # Sort possible letters by their frequency
        sorted_letters = sorted(possible_letters, key=lambda x: x[1], reverse=True)
        return [letter for letter, _ in sorted_letters]
    # def refine_with_ngrams(self, word):
    #     """
    #     Refine predictions using n-gram probabilities.
    #     """
    #     possible_letters = {}
    #     for i, char in enumerate(word):
    #         if char == "_":
    #             left_context = word[max(0, i - 1):i]  # Previous letter
    #             for letter in "abcdefghijklmnopqrstuvwxyz":
    #                 ngram = left_context + letter
    #                 possible_letters[letter] = possible_letters.get(letter, 0) + self.ngram_probs.get(ngram, 0)
        
    #     # Normalize probabilities
    #     total_prob = sum(possible_letters.values())
    #     if total_prob > 0:
    #         for letter in possible_letters:
    #             possible_letters[letter] /= total_prob
        
    #     return possible_letters


    def should_prefer_vowels(self):
        guessed_vowels = [l for l in self.guessed_letters if l in "aeiou"]
        return len(guessed_vowels) < 3

    def fallback_guess(self, word):
        """
        Fallback mechanism using vowel prior probabilities based on word length.
        """
        # Determine the length of the word
        word_length = len(word.replace("_", ""))
    
        # Get the vowel prior probabilities for the word length
        vowels = "aeiou"
        vowel_probs = self.vowel_prior.get(word_length, {})
    
        # Filter out already guessed vowels
        remaining_vowels = [v for v in vowels if v not in self.guessed_letters]
        
        if remaining_vowels:
            # Choose the vowel with the highest prior probability
            guess_letter = max(remaining_vowels, key=lambda v: vowel_probs.get(v, 0))
            self.guessed_letters.append(guess_letter)
            return guess_letter
    
        # Fallback to frequency-based guess if no vowels are available
        remaining_letters = [chr(i + ord('a')) for i in range(26) if chr(i + ord('a')) not in self.guessed_letters]
        letter_counts = collections.Counter("".join(self.full_dictionary))
        guess_letter = max(remaining_letters, key=lambda l: letter_counts.get(l, 0))
        self.guessed_letters.append(guess_letter)
        return guess_letter

    ##########################################################
    # You'll likely not need to modify any of the code below #
    ##########################################################
    
    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location, "r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary
                
    def start_game(self, practice=True, verbose=True):
        # Reset guessed letters and current plausible dictionary
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary
                         
        response = self.request("/new_game", {"practice": practice})
        if response.get('status') == "approved":
            game_id = response.get('game_id')
            word = response.get('word')
            tries_remains = response.get('tries_remains')
            if verbose:
                print(f"Successfully started a new game! Game ID: {game_id}. # of tries remaining: {tries_remains}. Word: {word}.")
            while tries_remains > 0:
                guess_letter = self.guess(word, tries_remains)  # Get guessed letter from the guess function
                if verbose:
                    print(f"Guessing letter: {guess_letter}")
                    
                # Append guessed letter to guessed letters list
                self.guessed_letters.append(guess_letter)
                
                try:
                    res = self.request("/guess_letter", {"request": "guess_letter", "game_id": game_id, "letter": guess_letter})
                except HangmanAPIError:
                    print('HangmanAPIError exception caught on request.')
                    continue
                except Exception as e:
                    print('Other exception caught on request.')
                    raise e
               
                if verbose:
                    print(f"Server response: {res}")
                
                status = res.get('status')
                tries_remains = res.get('tries_remains')
                if status == "success":
                    if verbose:
                        print(f"Successfully finished game: {game_id}")
                    return True
                elif status == "failed":
                    reason = res.get('reason', '# of tries exceeded!')
                    if verbose:
                        print(f"Failed game: {game_id}. Reason: {reason}")
                    return False
                elif status == "ongoing":
                    word = res.get('word')
        else:
            if verbose:
                print("Failed to start a new game")
        return False

    def my_status(self):
        return self.request("/my_status", {})

    def request(self, path, args=None, post_args=None, method=None):
        if args is None:
            args = dict()
        if post_args is not None:
            method = "POST"

        if self.access_token:
            if post_args and "access_token" not in post_args:
                post_args["access_token"] = self.access_token
            elif "access_token" not in args:
                args["access_token"] = self.access_token

        time.sleep(0.2)
        num_retry, time_sleep = 50, 2
        for it in range(num_retry):
            try:
                response = self.session.request(
                    method or "GET",
                    self.hangman_url + path,
                    timeout=self.timeout,
                    params=args,
                    data=post_args,
                    verify=False
                )
                break
            except requests.HTTPError as e:
                response = json.loads(e.read())
                raise HangmanAPIError(response)
            except requests.exceptions.SSLError as e:
                if it + 1 == num_retry:
                    raise
                time.sleep(time_sleep)

        headers = response.headers
        if 'json' in headers['content-type']:
            result = response.json()
        elif "access_token" in parse_qs(response.text):
            query_str = parse_qs(response.text)
            if "access_token" in query_str:
                result = {"access_token": query_str["access_token"][0]}
                if "expires" in query_str:
                    result["expires"] = query_str["expires"][0]
            else:
                raise HangmanAPIError(response.json())
        else:
            raise HangmanAPIError('Maintype was not text, or querystring')

        if result and isinstance(result, dict) and result.get("error"):
            raise HangmanAPIError(result)
        return result

class HangmanAPIError(Exception):
    def __init__(self, result):
        self.result = result
        self.code = None
        try:
            self.type = result["error_code"]
        except (KeyError, TypeError):
            self.type = ""

        try:
            self.message = result["error_description"]
        except (KeyError, TypeError):
            try:
                self.message = result["error"]["message"]
                self.code = result["error"].get("code")
                if not self.type:
                    self.type = result["error"].get("type", "")
            except (KeyError, TypeError):
                try:
                    self.message = result["error_msg"]
                except (KeyError, TypeError):
                    self.message = result

        Exception.__init__(self, self.message)

In [ ]:
# ## ML model only for long words + Ngram filtering and consonant based refining for shorter words
# import re
# import collections
# import pickle
# import time
# import numpy as np
# from xgboost import XGBClassifier
# from lightgbm import LGBMClassifier
# from catboost import CatBoostClassifier
# import requests


# class HangmanAI2:
#     def __init__(self, access_token=None, session=None, timeout=None):
#         self.hangman_url = self.determine_hangman_url()
#         self.access_token = access_token
#         self.session = session or requests.Session()
#         self.timeout = timeout

#         self.guessed_letters = []

#         full_dictionary_location = "/kaggle/input/words-250000/words_250000_train.txt"
#         self.full_dictionary = self.build_dictionary(full_dictionary_location)
#         self.vowel_prior = self.initialize_vowel_prior()
#         self.ngram_probs = self.build_ngram_probabilities(self.full_dictionary)  # New line


#         # self.x, self.y = self.generate_data(self.full_dictionary)


#         self.current_dictionary = []
#         self.prev_values = [0, 0, 0]
#         self.guessed_letters = []

#         self.x, self.y = self.generate_data(self.full_dictionary)

#         # Initialize models
#         self.model_xgb = XGBClassifier(n_estimators=150, max_depth=9, subsample=0.7, colsample_bytree=0.7)
#         self.model_lgb = LGBMClassifier(num_leaves=50, n_estimators=200, min_child_weight=0.5, reg_alpha=0.01, reg_lambda=0.01)
#         self.model_cb = CatBoostClassifier(iterations=100, learning_rate=0.1, depth=2, loss_function='MultiClass', verbose=False)

#         # Train or load models
#         self.model_cb = self.training(self.model_cb, "catb_newc1.pkl")
#         self.model_xgb = self.training(self.model_xgb, "xgboost_newc1.pkl")
#         self.model_lgb = self.training(self.model_lgb, "lightgbm_newc1.pkl")

#     @staticmethod
#     def determine_hangman_url():
#         links = ['https://trexsim.com', 'https://sg.trexsim.com']
#         data = {link: 0 for link in links}
#         for link in links:
#             requests.get(link)
#             for i in range(10):
#                 start = time.time()
#                 requests.get(link)
#                 data[link] += time.time() - start
#         link = sorted(data.items(), key=lambda x: x[1])[0][0]
#         return f"{link}/trexsim/hangman"
#     def build_ngram_probabilities(self, dictionary, n=2):
#         """
#         Build n-gram probabilities from the word dictionary.
#         """
#         ngram_counts = collections.Counter()
#         total_counts = collections.Counter()

#         for word in dictionary:
#             for i in range(len(word) - n + 1):
#                 ngram = word[i:i + n]
#                 ngram_counts[ngram] += 1
#                 total_counts[word[i]] += 1

#         ngram_probs = {ngram: count / total_counts[ngram[0]] for ngram, count in ngram_counts.items()}
#         return ngram_probs
#     def refine_with_ngrams(self, word):
#         """
#         Refine predictions using n-gram probabilities.
#         """
#         possible_letters = {}
#         for i, char in enumerate(word):
#             if char == "_":
#                 left_context = word[max(0, i - 2):i]  # Look at the previous two characters
#                 for letter in "abcdefghijklmnopqrstuvwxyz":
#                     ngram = left_context + letter
#                     possible_letters[letter] = possible_letters.get(letter, 0) + self.ngram_probs.get(ngram, 0)
        
#         # Sort by probability and return top candidates as a list
#         sorted_letters = sorted(possible_letters.items(), key=lambda x: x[1], reverse=True)
#         final_candidates = [letter for letter, _ in sorted_letters]  # Return a list of letters based on sorted probabilities
    
#         return final_candidates


#     def build_dictionary(self, file_path):
#         with open(file_path, "r") as f:
#             return [line.strip() for line in f.readlines()]
#     def training(self, classifier, weight_file):
#         """Train or load the model"""
#         model_path = os.path.join("/kaggle/working", weight_file)
        
#         # Check if model file exists
#         if os.path.exists(model_path):
#             # If model exists, load it
#             with open(model_path, 'rb') as f:
#                 model = pickle.load(f)
#             print(f"Model loaded from {model_path}")
#         else:
#             # If model does not exist, train it
#             model = classifier.fit(self.x, self.y)
#             # Save the trained model
#             with open(model_path, 'wb') as f:
#                 pickle.dump(model, f)
#             print(f"Model trained and saved to {model_path}")

#         return model
#     def generate_data(self, dictionary):
#         """
#         Generate the feature set (self.x) and target labels (self.y) from the given word dictionary.
#         Features are one-hot encoded word variations, and targets are the index of missing letters.
        
#         Parameters:
#         - dictionary: List of words used for training the models.
        
#         Returns:
#         - x: Feature array (one-hot encoded variations of words).
#         - y: Label array (corresponding indices of missing letters).
#         """
#         x, y = [], []
#         for word in dictionary:
#             variations, missing_letters = self.create_variations(word)
#             for variation, missing_letter in zip(variations, missing_letters):
#                 # Step 1: Alphabet encoding of word variation
#                 alphabet_encoded_variation = self.alphabet_encoding(variation)
                
#                 # Step 2: One-hot encoding of alphabetic positions
#                 onehot_encoded_variation = self.onehot_encoding(alphabet_encoded_variation)
                
#                 # Append the feature and the label to x and y lists respectively
#                 x.append(onehot_encoded_variation)
                
#                 # The label is the index of the missing letter in the alphabet (0-25)
#                 y.append(ord(missing_letter) - ord('a'))
        
#         # Convert x and y into numpy arrays
#         return np.array(x), np.array(y)
    

#     def create_variations(self, word):
#         variations, letters = [], []
#         for i, char in enumerate(word):
#             variation = word[:i] + '_' + word[i+1:]
#             if variation not in variations:
#                 variations.append(variation)
#                 letters.append(char)
#         return variations, letters

#     def alphabet_encoding(self, word):
#         return [ord(letter) - ord('a') + 1 for letter in word]

#     def onehot_encoding(self, vector, max_len=30):
#         padded_vector = np.zeros(max_len, dtype=int)
#         padded_vector[:len(vector)] = vector
#         return padded_vector

#     def next_letter_prediction(self, encoding, model):
#         predictions = model.predict_proba(encoding.reshape(1, -1))
#         for i, prob in enumerate(predictions[0]):
#             if chr(i + ord('a')) in self.guessed_letters:
#                 predictions[0][i] = 0
#         return predictions

#     def probability_max(self, predictions):
#     # Weighted averaging of predictions
#         weights = [0.4, 0.3, 0.3]  # Example weights for CatBoost, XGBoost, and LightGBM
#         weighted_predictions = sum(weight * predictions[i][0] for i, weight in enumerate(weights))
#         return np.array(weighted_predictions)

#     def initialize_vowel_prior(self):
#         """
#         Initialize vowel prior probabilities based on word length.
#         """
#         vowel_counts = {length: collections.Counter() for length in range(1, 36)}  # Assuming max length of 35
#         vowels = "aeiou"

#         for word in self.full_dictionary:
#             word_length = len(word)
#             if word_length <= 35:
#                 for char in word:
#                     if char in vowels:
#                         vowel_counts[word_length][char] += 1

#         vowel_prior = {}
#         for length, counts in vowel_counts.items():
#             total = sum(counts.values())
#             if total > 0:
#                 vowel_prior[length] = {vowel: count / total for vowel, count in counts.items()}
#             else:
#                 vowel_prior[length] = {vowel: 0 for vowel in vowels}

#         return vowel_prior
#     def refine_with_common_consonants(self, word):
#         """
#         Refines the guess for short words (<= 6 letters) by focusing on common consonants.
#         """
#         # Common consonants based on language frequency
#         common_consonants = ['r', 't', 'n', 's', 'l', 'c', 'd', 'p', 'm', 'b', 'h', 'f', 'g', 'y']
    
#         # Remove letters that have already been guessed or are already present in the word
#         guessed_letters_set = set(self.guessed_letters)
#         existing_letters = set(word.replace("_", ""))
#         possible_consonants = [c for c in common_consonants if c not in guessed_letters_set and c not in existing_letters]
    
#         return possible_consonants


#     def guess(self, word, tries_remains):
#         word = word.replace(" ", "")
        
#         # Step 1: Check if word length is 6 or less
#         if len(word.replace("_", "")) <= 6:
#             # If word length is 6 or less, use only dictionary-based filtering and vowel prior
#             final_predictions = self.refine_with_dictionary(word)
    
#             # Step 2: If no letters are possible from the dictionary, use vowel prior or fallback guess
#             if not final_predictions:
#     # If no predictions, refine using common consonants and n-grams
#                 # common_consonants = self.refine_with_common_consonants(word)
#                 ngram_candidates = self.refine_with_ngrams(word)
            
#                 # Combine the common consonants and n-gram candidates
#                 final_predictions = ngram_candidates
                
#                 # If still no predictions after refinement, check if we should prefer vowels
#                 if not final_predictions:
#                     if self.should_prefer_vowels():
#                         return self.fallback_guess(word)
#                     else:
#                         # If fallback mechanism is not applicable, return the best guess based on frequency
#                         return self.fallback_guess(word)

#             # Select the most frequent candidate from the final predictions
#             guessed_letter = final_predictions[0]
#             self.guessed_letters.append(guessed_letter)
#             return guessed_letter

    
#         else:
#             # If word length is more than 6, use machine learning models (XGBoost, CatBoost, LightGBM)
#             encoding = self.onehot_encoding(self.alphabet_encoding(word))
    
#             # Get predictions from ML models
#             predictions = [
#                 self.next_letter_prediction(encoding, model)
#                 for model in [self.model_cb, self.model_xgb, self.model_lgb]
#             ]
            
#             final_predictions = self.probability_max(predictions)
    
#             # Step 3: Refine predictions using dictionary
#             possible_letters = self.refine_with_dictionary(word)
#             if possible_letters:
#                 for i in range(26):
#                     if chr(i + ord('a')) not in possible_letters:
#                         final_predictions[i] = 0
    
#             # Step 4: Normalize final predictions
#             final_predictions /= final_predictions.sum()
    
#             # Step 5: Integrate heuristics - Vowel preference if tries remaining > 4
#             if self.should_prefer_vowels():
#                 for vowel in "aeiou":
#                     idx = ord(vowel) - ord('a')
#                     final_predictions[idx] *= 1.5
    
#             # Step 6: Backup mechanism if no predictions
#             if final_predictions.sum() == 0:
#                 return self.fallback_guess(word)
    
#             # Step 7: Select the best guess
#             guessed_letter = chr(final_predictions.argmax() + ord('a'))
#             self.guessed_letters.append(guessed_letter)
#             return guessed_letter


#     def refine_with_dictionary(self, word):
#         pattern = "^" + word.replace("_", ".") + "$"
#         refined_dict = [w for w in self.full_dictionary if re.match(pattern, w)]
        
#         # Score candidates based on frequency
#         letter_frequencies = collections.Counter("".join(refined_dict))
#         possible_letters = set()
#         for candidate in refined_dict:
#             for char in candidate:
#                 if char not in self.guessed_letters:
#                     possible_letters.add((char, letter_frequencies[char]))
        
#         # Sort possible letters by their frequency
#         sorted_letters = sorted(possible_letters, key=lambda x: x[1], reverse=True)
#         return [letter for letter, _ in sorted_letters]
#     # def refine_with_ngrams(self, word):
#     #     """
#     #     Refine predictions using n-gram probabilities.
#     #     """
#     #     possible_letters = {}
#     #     for i, char in enumerate(word):
#     #         if char == "_":
#     #             left_context = word[max(0, i - 1):i]  # Previous letter
#     #             for letter in "abcdefghijklmnopqrstuvwxyz":
#     #                 ngram = left_context + letter
#     #                 possible_letters[letter] = possible_letters.get(letter, 0) + self.ngram_probs.get(ngram, 0)
        
#     #     # Normalize probabilities
#     #     total_prob = sum(possible_letters.values())
#     #     if total_prob > 0:
#     #         for letter in possible_letters:
#     #             possible_letters[letter] /= total_prob
        
#     #     return possible_letters


#     def should_prefer_vowels(self):
#         guessed_vowels = [l for l in self.guessed_letters if l in "aeiou"]
#         return len(guessed_vowels) < 3

#     def fallback_guess(self, word):
#         """
#         Fallback mechanism using vowel prior probabilities based on word length.
#         """
#         # Determine the length of the word
#         word_length = len(word.replace("_", ""))
    
#         # Get the vowel prior probabilities for the word length
#         vowels = "aeiou"
#         vowel_probs = self.vowel_prior.get(word_length, {})
    
#         # Filter out already guessed vowels
#         remaining_vowels = [v for v in vowels if v not in self.guessed_letters]
        
#         if remaining_vowels:
#             # Choose the vowel with the highest prior probability
#             guess_letter = max(remaining_vowels, key=lambda v: vowel_probs.get(v, 0))
#             self.guessed_letters.append(guess_letter)
#             return guess_letter
    
#         # Fallback to frequency-based guess if no vowels are available
#         remaining_letters = [chr(i + ord('a')) for i in range(26) if chr(i + ord('a')) not in self.guessed_letters]
#         letter_counts = collections.Counter("".join(self.full_dictionary))
#         guess_letter = max(remaining_letters, key=lambda l: letter_counts.get(l, 0))
#         self.guessed_letters.append(guess_letter)
#         return guess_letter

#     ##########################################################
#     # You'll likely not need to modify any of the code below #
#     ##########################################################
    
#     def build_dictionary(self, dictionary_file_location):
#         text_file = open(dictionary_file_location, "r")
#         full_dictionary = text_file.read().splitlines()
#         text_file.close()
#         return full_dictionary
                
#     def start_game(self, practice=True, verbose=True):
#         # Reset guessed letters and current plausible dictionary
#         self.guessed_letters = []
#         self.current_dictionary = self.full_dictionary
                         
#         response = self.request("/new_game", {"practice": practice})
#         if response.get('status') == "approved":
#             game_id = response.get('game_id')
#             word = response.get('word')
#             tries_remains = response.get('tries_remains')
#             if verbose:
#                 print(f"Successfully started a new game! Game ID: {game_id}. # of tries remaining: {tries_remains}. Word: {word}.")
#             while tries_remains > 0:
#                 guess_letter = self.guess(word, tries_remains)  # Get guessed letter from the guess function
#                 if verbose:
#                     print(f"Guessing letter: {guess_letter}")
                    
#                 # Append guessed letter to guessed letters list
#                 self.guessed_letters.append(guess_letter)
                
#                 try:
#                     res = self.request("/guess_letter", {"request": "guess_letter", "game_id": game_id, "letter": guess_letter})
#                 except HangmanAPIError:
#                     print('HangmanAPIError exception caught on request.')
#                     continue
#                 except Exception as e:
#                     print('Other exception caught on request.')
#                     raise e
               
#                 if verbose:
#                     print(f"Server response: {res}")
                
#                 status = res.get('status')
#                 tries_remains = res.get('tries_remains')
#                 if status == "success":
#                     if verbose:
#                         print(f"Successfully finished game: {game_id}")
#                     return True
#                 elif status == "failed":
#                     reason = res.get('reason', '# of tries exceeded!')
#                     if verbose:
#                         print(f"Failed game: {game_id}. Reason: {reason}")
#                     return False
#                 elif status == "ongoing":
#                     word = res.get('word')
#         else:
#             if verbose:
#                 print("Failed to start a new game")
#         return False

#     def my_status(self):
#         return self.request("/my_status", {})

#     def request(self, path, args=None, post_args=None, method=None):
#         if args is None:
#             args = dict()
#         if post_args is not None:
#             method = "POST"

#         if self.access_token:
#             if post_args and "access_token" not in post_args:
#                 post_args["access_token"] = self.access_token
#             elif "access_token" not in args:
#                 args["access_token"] = self.access_token

#         time.sleep(0.2)
#         num_retry, time_sleep = 50, 2
#         for it in range(num_retry):
#             try:
#                 response = self.session.request(
#                     method or "GET",
#                     self.hangman_url + path,
#                     timeout=self.timeout,
#                     params=args,
#                     data=post_args,
#                     verify=False
#                 )
#                 break
#             except requests.HTTPError as e:
#                 response = json.loads(e.read())
#                 raise HangmanAPIError(response)
#             except requests.exceptions.SSLError as e:
#                 if it + 1 == num_retry:
#                     raise
#                 time.sleep(time_sleep)

#         headers = response.headers
#         if 'json' in headers['content-type']:
#             result = response.json()
#         elif "access_token" in parse_qs(response.text):
#             query_str = parse_qs(response.text)
#             if "access_token" in query_str:
#                 result = {"access_token": query_str["access_token"][0]}
#                 if "expires" in query_str:
#                     result["expires"] = query_str["expires"][0]
#             else:
#                 raise HangmanAPIError(response.json())
#         else:
#             raise HangmanAPIError('Maintype was not text, or querystring')

#         if result and isinstance(result, dict) and result.get("error"):
#             raise HangmanAPIError(result)
#         return result

# class HangmanAPIError(Exception):
#     def __init__(self, result):
#         self.result = result
#         self.code = None
#         try:
#             self.type = result["error_code"]
#         except (KeyError, TypeError):
#             self.type = ""

#         try:
#             self.message = result["error_description"]
#         except (KeyError, TypeError):
#             try:
#                 self.message = result["error"]["message"]
#                 self.code = result["error"].get("code")
#                 if not self.type:
#                     self.type = result["error"].get("type", "")
#             except (KeyError, TypeError):
#                 try:
#                     self.message = result["error_msg"]
#                 except (KeyError, TypeError):
#                     self.message = result

#         Exception.__init__(self, self.message)

In [ ]:
!pip install tqdm


In [4]:
api1 = HangmanAI1(access_token="b10d926d7c581f8b023c673a54bf4f", timeout=2000)


Model loaded from /kaggle/input/gboosting/pytorch/default/1/catb_newc1.pkl
Model loaded from /kaggle/input/gboosting/pytorch/default/1/xgboost_newc1.pkl
Model loaded from /kaggle/input/gboosting/pytorch/default/1/lightgbm_newc1.pkl


In [ ]:
# api2 = HangmanAI2(access_token="b10d926d7c581f8b023c673a54bf4f", timeout=2000)


In [7]:
api1.start_game(practice=1,verbose=True)
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api1.my_status() # Get my game stats: (# of tries, # of wins)
practice_success_rate = (total_practice_successes-242) / (total_practice_runs-644)
print('run %d practice games out of an allotted 100,000. total practice successes are %d. practice success rate so far = %.3f' % (total_practice_runs,total_practice_successes, practice_success_rate))

/opt/conda/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'trexsim.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Successfully started a new game! Game ID: 63f0b3e4a204. # of tries remaining: 6. Word: _ _ _ _ _ _ _ _ _ _ .
Guessing letter: e


/opt/conda/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'trexsim.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Server response: {'game_id': '63f0b3e4a204', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ _ e e _ _ e _ '}
Guessing letter: a


/opt/conda/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'trexsim.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


...

/opt/conda/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'trexsim.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Server response: {'game_id': '63f0b3e4a204', 'status': 'ongoing', 'tries_remains': 3, 'word': 's t o _ e e a r e d '}
Guessing letter: n


/opt/conda/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'trexsim.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Server response: {'game_id': '63f0b3e4a204', 'status': 'success', 'tries_remains': 3, 'word': 's t o n e e a r e d '}
Successfully finished game: 63f0b3e4a204
run 647 practice games out of an allotted 100,000. total practice successes are 245. practice success rate so far = 1.000


/opt/conda/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'trexsim.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [ ]:
# api2.start_game(practice=1,verbose=True)
# [total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api2.my_status() # Get my game stats: (# of tries, # of wins)
# practice_success_rate = (total_practice_successes-215) / (total_practice_runs-566)
# print('run %d practice games out of an allotted 100,000. total practice successes are %d. practice success rate so far = %.3f' % (total_practice_runs,total_practice_successes, practice_success_rate))

In [9]:
from tqdm import tqdm
import warnings
import time

In [10]:
warnings.filterwarnings("ignore")

for i in tqdm(range(1000), desc="Playing Games"):
    print('Playing ', i, ' th game')
    # Uncomment the following line to execute your final runs. Do not do this until you are satisfied with your submission
    api1.start_game(practice=0,verbose=False)
    
    # DO NOT REMOVE as otherwise the server may lock you out for too high frequency of requests
    time.sleep(0.5)

Playing Games:   0%|          | 0/1000 [00:00<?, ?it/s]

Playing  0  th game


Playing Games:   0%|          | 1/1000 [00:07<1:57:12,  7.04s/it]

Playing  1  th game


Playing Games:   0%|          | 2/1000 [00:11<1:34:26,  5.68s/it]

...

Playing  551  th game


Playing Games:  55%|█████▌    | 552/1000 [47:07<36:15,  4.86s/it]

Playing  552  th game


Playing Games:  55%|█████▌    | 552/1000 [47:11<38:17,  5.13s/it]


HangmanAPIError: {'error': 'You have reached 1000 of games', 'status': 'denied'}

In [11]:
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api1.my_status() # Get my game stats: (# of tries, # of wins)
success_rate = total_recorded_successes/total_recorded_runs
print('overall success rate = %.3f' % success_rate)

overall success rate = 0.395
